In [ ]:
#20250514 Wed, jeri
"""
출력 방식 변경: 조회수, 업로드일
추가한 내용: 상품 조회, 영상의 더보기 설명란

250514
한 일 : 더보기란 클릭 코드, 더보기란에 있는 설명들 크롤링 완료
해야 할 일 : 조회수, 업로드일, 제품 개수를 크롤링 못하는 중(해결해야 함), 제품 정보 가져와야 함(이미지, 제품명, 판매처, 구매 링크)
"""
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
import os
import pandas as pd
from datetime import datetime
# 버전 관리를 위해 import
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#더보기란에 있는 조회수, 날짜, 상품 개수를 위해 import
from bs4 import BeautifulSoup
import re


# 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized') # 유튜브 더보기 클릭 및 정보 가져오기

# 크롬 드라이버 실행(자동 버전 관리)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 드라이버가 실행될 영상 url
video_id = 'w7uwgwj6aZM'
video_url = f'https://www.youtube.com/watch?v={video_id}'
driver.get(video_url)

# 페이지 로딩 대기
time.sleep(5)

# 오늘 날짜 문자열 (YYMMDD)
today_str = datetime.today().strftime('%y%m%d')

"""
===============
"더보기" 버튼 클릭
===============
"""
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "description-inline-expander"))
    )
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, 'tp-yt-paper-button#expand').click()
    time.sleep(1)
except Exception as e:
    print("더보기 버튼 클릭 실패:", e)

"""
=====================
더보기란에 있는 텍스트 추출
=====================
"""
try:
    description = driver.find_element(By.ID, 'description-inline-expander').text
except:
    print("설명란 추출 실패")
    description = "설명 없음"

"""
=======================
채널명, 영상 제목, 구독자 수
=======================
"""
# 영상에서 들고온 정보들을 담을 딕셔너리
base_info_results = {}

base_info = {
    "channel": '//*[@id="text"]/a',
    "title": '//*[@id="title"]/h1',
    "subscriber": '//*[@id="owner-sub-count"]',
}

"""
==================================================
조회수, 업로드일, 제품 수 추출 (더보기 설명란에서 정규표현식으로)
==================================================
"""

soup = BeautifulSoup(driver.page_source, 'html.parser')
spans = soup.select('yt_formatted_string#info > span')

# 공백 제외하고 실제 텍스트만 추출
info_texts = [span.get_text(strip=True) for span in spans if span.get_text(strip=True)]

# 조회수 숫자만 추출 (쉼표 유지)
match_views = re.search(r'조회수\s*([\d,]+)', info_text)
youtube_views = match_views.group(1) if match_views else "조회수 정보 못 찾음"

# 날짜 변환: "2025. 5. 12." → "2025-05-12"
upload_date = "업로드 날짜 정보 못 찾음"
match_date_dot = re.search(r'(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})\.', info_text)
match_date_kor = re.search(r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일', info_text)

if match_date_dot:
    y, m, d = match_date_dot.groups()
    upload_date = f"{int(y):04}-{int(m):02}-{int(d):02}"
elif match_date_kor:
    y, m, d = match_date_kor.groups()
    upload_date = f"{int(y):04}-{int(m):02}-{int(d):02}"

# 제품 수
match_product = re.search(r'(\d+)개 제품', info_text)
product_count = int(match_product.group(1)) if match_product else "제품 정보 못 찾음"

"""
=======================
구독자 수 표기 방식 변환 함수
=======================
"""
def parse_subscriber_count(text):
    """
    '구독자 281만명' → '2,810,000'
    '구독자 3.45천명' → '3,450'
    '구독자 102명' → '102'
    """
    match = re.search(r'([\d\.]+)([천만]?)명', text)
    if not match:
        return "0"

    number = float(match.group(1))
    unit = match.group(2)

    if unit == '천':
        number *= 1_000
    elif unit == '만':
        number *= 10_000

    return f"{int(number):,}"  # 쉼표 포함된 문자열로 반환

for bases, xpath in base_info.items():
    try:
        # 로딩될 때까지 최대 60초 기다림
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        text = element.text
        if bases == "subscriber":
            base_info_results[bases] = parse_subscriber_count(text)
        else:
            base_info_results[bases] = text
            
    except Exception as e:
        print(f"Error fetching {bases}: {e}")
        base_info_results[bases] = "Not found"

"""
=======================
유튜브 링크, 추출일 컬럼 추가
=======================
"""
base_info_results['영상 링크'] = video_url
base_info_results['추출일'] = today_str
base_info_results['조회수'] = youtube_views
base_info_results['업로드일'] = youtube_upload_date
base_info_results['포함된 제품'] = youtube_product
base_info_results['더보기란 설명'] = description

"""
=========
컬럼명 매핑
=========
"""
column_name_mapping = {
    "channel": "채널명",
    "title": "영상 제목",
    "subscriber": "구독자 수",
}

"""
=========================
원하는 컬럼 순서를 리스트로 지정
=========================
"""
column_order = [
    '추출일', '채널명', '영상 제목', '영상 링크', '구독자 수',
    '조회수', '업로드일', '포함된 제품', '더보기란 설명'
    ]

# DataFrame 생성 및 컬럼명 변경
df = pd.DataFrame([base_info_results])
df = df.rename(columns=column_name_mapping)
# 원하는 컬럼 순서로 재정렬
df = df[column_order]

"""
==============
base info 출력
==============
"""
print("\n=== Base Info ===")
for key, value in base_info_results.items():
    print(f"{column_name_mapping.get(key, key)}: {value}")
# 더보기란 출력
print(description, youtube_views, youtube_upload_date, youtube_product)

"""
=================
파일명 중복 방지 저장
=================
"""
base_filename = f'침착맨_유튜브_{today_str}'
ext = '.xlsx'
filename = f'{base_filename}{ext}'
counter = 1
while os.path.exists(filename):
    filename = f'{base_filename}_{counter}{ext}'
    counter += 1

df.to_excel(filename, index=False)
print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

"""
==========
드라이버 종료
==========
"""
try:
    if driver.session_id:
        _ = BeautifulSoup(driver.page_source, 'html.parser')
except Exception as e:
    print(f"브라우저 세션 오류: {e}")

driver.quit()




=== Base Info ===
채널명: 침착맨
영상 제목: 버거킹, 우리의 아이디어를 꼭 들어주시기 바랍니다
구독자 수: 2,810,000
영상 링크: https://www.youtube.com/watch?v=w7uwgwj6aZM
추출일: 250514
조회수: 조회수 정보 못 찾음
업로드일: 업로드 날짜 못 찾음
포함된 제품: 제품 정보 못 찾음
더보기란 설명: 이 영상은 [버거킹]의 유료 광고가 포함되어 있습니다.

▶같이 보면 좋은 추천 영상
   • 담 걸린 침착맨의 아이언맨 맛 버거 먹방  
▶'침착맨의 일상재롱' 모아보기
   • 침착맨의 일상재롱  

▶이 영상의 생방송 원본
  •2025년 4월 28일 방송분:    • 2025년 04월 28일 1부 | 버거킹 신메뉴 크리스퍼 2종 먹방  

▶출연: 침착맨
▶편집: 15수자
▶썸네일: 15수자 

▶협업제안: info@chimchakman.com

#침착맨 #버거킹 #크리스퍼 #크리스퍼클래식 #크리스퍼클래식BLT #치킨버거 #IDONTLIKEWHOPPER #ILOVEKRISPPER #노윤서 #추영우
음악
노래 1곡
Feel The Funk
Jimmy Fontanez/Media Right Productions
Feel The Funk
음악
스크립트
스크립트를 보면서 시청하세요.
스크립트 표시
침착맨
구독자 281만명
동영상
정보
토크 영상은? 침착맨
게임 영상은? 침착맨 플러스
생방송은? 침착맨 원본 박물관
침착맨 공식 인스타그램
간략히
이 영상은 [버거킹]의 유료 광고가 포함되어 있습니다.

▶같이 보면 좋은 추천 영상
   • 담 걸린 침착맨의 아이언맨 맛 버거 먹방  
▶'침착맨의 일상재롱' 모아보기
   • 침착맨의 일상재롱  

▶이 영상의 생방송 원본
  •2025년 4월 28일 방송분:    • 2025년 04월 28일 1부 | 버거킹 신메뉴 크리스퍼 2종 먹방  

▶출연: 침착맨
▶편집: 15수자
▶썸네일: 15수자 

▶협업제안: info@chimchakman.com

#침착맨